### Disclaimer
- notebook first made by naver-boostcamp
- First edited by cuffyluv - 251208

https://stages.ai/competitions/378/board/community/post/2970

In [4]:
import json
import numpy as np
from collections import Counter
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

In [3]:
# load json annotation = {dataset file 경로}

annotation = "../../dataset/train.json"
with open(annotation) as f: 
    data = json.load(f)

var = [(ann['image_id'], ann['category_id']) for ann in data['annotations']]
X = np.ones((len(data['annotations']),1))
y = np.array([v[1] for v in var])
groups = np.array([v[0] for v in var])

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=411)

for train_idx, val_idx in cv.split(X, y, groups):
    print("TRAIN:", groups[train_idx])
    print(" ", y[train_idx])
    print(" TEST:", groups[val_idx])
    print(" ", y[val_idx])

TRAIN: [   0    1    1 ... 4881 4881 4881]
  [0 3 7 ... 7 1 7]
 TEST: [   6   13   13 ... 4882 4882 4882]
  [1 6 7 ... 0 1 1]
TRAIN: [   0    1    1 ... 4882 4882 4882]
  [0 3 7 ... 0 1 1]
 TEST: [   5    5    5 ... 4876 4876 4878]
  [7 0 0 ... 0 2 0]
TRAIN: [   0    3    3 ... 4882 4882 4882]
  [0 2 6 ... 0 1 1]
 TEST: [   1    1    1 ... 4877 4877 4880]
  [3 7 4 ... 7 7 0]
TRAIN: [   1    1    1 ... 4882 4882 4882]
  [3 7 4 ... 0 1 1]
 TEST: [   0    3    3 ... 4881 4881 4881]
  [0 2 6 ... 7 1 7]
TRAIN: [   0    1    1 ... 4882 4882 4882]
  [0 3 7 ... 0 1 1]
 TEST: [   4    4    4 ... 4868 4872 4872]
  [1 1 1 ... 2 4 6]


In [6]:

# check distribution

def get_distribution(y):
    y_distr = Counter(y)
    y_vals_sum = sum(y_distr.values())
    return [f'{y_distr[i]/y_vals_sum:.2%}' for i in range(np.max(y) + 1)]

distrs = [get_distribution(y)]
index = ['training set']

for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    train_y, val_y = y[train_idx], y[val_idx]
    train_gr, val_gr = groups[train_idx], groups[val_idx]

    # 그룹 겹침 확인
    assert len(set(train_gr) & set(val_gr)) == 0

    distrs.append(get_distribution(train_y))
    distrs.append(get_distribution(val_y))
    index.append(f'train - fold{fold_ind}')
    index.append(f'val - fold{fold_ind}')

categories = [d['name'] for d in data['categories']]
df = pd.DataFrame(distrs, index=index, columns=[categories[i] for i in range(np.max(y) + 1)])

print(df)

              General trash   Paper Paper pack  Metal  Glass Plastic  \
training set         17.14%  27.45%      3.88%  4.04%  4.24%  12.72%   
train - fold0        16.96%  27.45%      3.79%  4.13%  4.48%  12.61%   
val - fold0          17.85%  27.42%      4.23%  3.70%  3.26%  13.15%   
train - fold1        17.14%  27.24%      4.01%  3.98%  4.28%  12.77%   
val - fold1          17.12%  28.17%      3.41%  4.26%  4.12%  12.51%   
train - fold2        17.31%  27.39%      3.83%  4.08%  4.13%  12.80%   
val - fold2          16.42%  27.68%      4.05%  3.88%  4.70%  12.36%   
train - fold3        17.30%  27.47%      3.87%  4.06%  4.22%  12.63%   
val - fold3          16.50%  27.36%      3.88%  3.99%  4.33%  13.07%   
train - fold4        16.97%  27.67%      3.88%  3.97%  4.10%  12.77%   
val - fold4          17.84%  26.48%      3.85%  4.38%  4.84%  12.50%   

              Styrofoam Plastic bag Battery Clothing  
training set      5.46%      22.37%   0.69%    2.02%  
train - fold0     5.51%  